<a href="https://colab.research.google.com/github/rieioa/Lab2_04/blob/master/2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/MyDrive/SCPC')

## Import

In [ ]:
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

import clip

In [ ]:
import os
import re
import torch
import random
import warnings
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
from transformers import Blip2Processor, Blip2ForConditionalGeneration

# 환경 설정
warnings.filterwarnings("ignore")
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("✅ Using device:", device)

✅ Using device: cuda


## Fixed RandomSeed

In [ ]:
# 시드 고정
def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything()

## Load Pre-trained Model

In [ ]:
# 모델 로딩
processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")
model = Blip2ForConditionalGeneration.from_pretrained(
    "Salesforce/blip2-opt-2.7b",
    device_map="auto",
    torch_dtype=torch.float16
)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/882 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/10.0G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/141 [00:00<?, ?B/s]

In [ ]:
# CLIP model
model, preprocess = clip.load("ViT-B/32", device=device)
CSV_PATH = "./dev_test.csv"
SUBMISSION_PATH = "sample_submission.csv"

df = pd.read_csv(CSV_PATH)
submission = pd.read_csv(SUBMISSION_PATH)

all_data = []  # 전체 이미지에 대한 question+answer 리스트 모음

for _, row in df.iterrows():
    index = row["ID"]
    image_path = row["img_path"]
    question = row["Question"]
    choices = [row["A"], row["B"], row["C"], row["D"]]
    texts = [f"{question} {choice}" for choice in choices]

    all_data.append({
        "ID": index,
        "images": image_path,
        "texts": texts
    })

predictions = []

for item in all_data:
    image = preprocess(Image.open(item["images"]).convert("RGB")).unsqueeze(0).to(device)
    text_tokens = clip.tokenize(item["texts"]).to(device)
    with torch.no_grad():
        image_features = model.encode_image(image)
        text_features = model.encode_text(text_tokens)
        image_features /= image_features.norm(dim=-1, keepdim=True)
        text_features /= text_features.norm(dim=-1, keepdim=True)
        similarity = image_features @ text_features.T
        pred_index = similarity.argmax().item()
        pred_label = ["A", "B", "C", "D"][pred_index]
    predictions.append(pred_label)

submission["answer"] = predictions
submission.to_csv("clip_submission.csv", index=False)
print("✅ clip_submission.csv 저장 완료!")

100%|███████████████████████████████████████| 338M/338M [00:08<00:00, 41.3MiB/s]


## Inference

In [ ]:
# 정답 알파벳 추출 함수
def extract_answer_letter(text):
    match = re.search(r"Answer:\s*([A-Da-d])\b", text)
    return match.group(1).upper() if match else "?"

In [ ]:
# 추론
test = pd.read_csv('test.csv')
results = []

for _, row in tqdm(test.iterrows(), total=len(test)):
    image = Image.open(row['img_path']).convert("RGB")
    choices = [row[c] for c in ['A', 'B', 'C', 'D']]

    prompt = (
        "You are a helpful AI that answers multiple-choice questions based on the given image.\n"
        "Select the best answer from A, B, C, or D.\n\n"
        f"Question: {row['Question']}\n"
        + "\n".join([f"{chr(65+i)}. {choice}" for i, choice in enumerate(choices)]) +
        "\nAnswer:"
    )

    inputs = processor(images=image, text=prompt, return_tensors="pt")
    inputs = {k: (v.half().to(device) if v.dtype == torch.float32 else v.to(device)) for k, v in inputs.items()}

    output = model.generate(**inputs, max_new_tokens=3, do_sample=False, temperature=0.0)
    decoded = processor.tokenizer.decode(output[0], skip_special_tokens=True).strip()
    results.append(extract_answer_letter(decoded))
print('✅ Done.')

100%|██████████| 852/852 [08:57<00:00,  1.58it/s]

✅ Done.


## Submission

In [ ]:
submission = pd.read_csv('./sample_submission.csv')
submission['answer'] = results
submission.to_csv('./baseline_submit.csv', index=False)
print("✅ Done.")

✅ Done.
